# DominiSigns - Extracción de Señas con MediaPipe

Este notebook permite procesar los videos del diccionario para extraer los puntos claves (poses) usando la GPU/CPU de Google Colab.

## 1. Configuración

In [ ]:
# Instalar dependencias
!pip install mediapipe opencv-python numpy

In [ ]:
import cv2
import mediapipe as mp
import json
import os
import glob
import numpy as np
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

## 2. Definir Rutas
Asegúrate de subir tu carpeta `DominiSigns` a tu Google Drive.

In [ ]:
# Ajusta esta ruta según donde hayas subido los archivos en tu Drive
BASE_DIR = '/content/drive/MyDrive/DominiSigns'
VIDEO_DIR = os.path.join(BASE_DIR, 'data', 'videos')
OUTPUT_DIR = os.path.join(BASE_DIR, 'data', 'pose_data')

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Leyendo videos de: {VIDEO_DIR}")
print(f"Guardando datos en: {OUTPUT_DIR}")

## 3. Script de Procesamiento

In [ ]:
mp_holistic = mp.solutions.holistic

def process_video(video_path):
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    # Obtener categoría (carpeta padre)
    category = os.path.basename(os.path.dirname(video_path))
    
    # Crear carpeta de categoría en output si no existe
    category_out_dir = os.path.join(OUTPUT_DIR, category)
    os.makedirs(category_out_dir, exist_ok=True)
    
    output_path = os.path.join(category_out_dir, f"{video_name}.json")
    
    if os.path.exists(output_path):
        print(f"Saltando {video_name}, ya procesado.")
        return

    cap = cv2.VideoCapture(video_path)
    frames_data = []
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    with mp_holistic.Holistic(
        static_image_mode=False,
        model_complexity=2,
        enable_segmentation=False,
        refine_face_landmarks=True) as holistic:
        
        frame_idx = 0
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break
                
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)
            
            frame_entry = {
                'frame': frame_idx,
                'timestamp': frame_idx / fps if fps > 0 else 0,
                'pose': [],
                'left_hand': [],
                'right_hand': [],
                'face': []
            }
            
            if results.pose_landmarks:
                frame_entry['pose'] = [{'x': l.x, 'y': l.y, 'z': l.z, 'visibility': l.visibility} for l in results.pose_landmarks.landmark]
            if results.left_hand_landmarks:
                frame_entry['left_hand'] = [{'x': l.x, 'y': l.y, 'z': l.z} for l in results.left_hand_landmarks.landmark]
            if results.right_hand_landmarks:
                frame_entry['right_hand'] = [{'x': l.x, 'y': l.y, 'z': l.z} for l in results.right_hand_landmarks.landmark]
                
            if frame_entry['pose'] or frame_entry['left_hand'] or frame_entry['right_hand']:
                frames_data.append(frame_entry)
                
            frame_idx += 1
            
    cap.release()
    
    with open(output_path, 'w') as f:
        json.dump({'fps': fps, 'frames': frames_data}, f)
        
    print(f"Procesado {category}/{video_name}: {len(frames_data)} frames")

In [ ]:
import glob
video_files = glob.glob(os.path.join(VIDEO_DIR, '**/*.mp4'), recursive=True)
print(f"Encontrados {len(video_files)} videos.")

for video in video_files:
    try:
        process_video(video)
    except Exception as e:
        print(f"Error en {video}: {e}")